In [0]:
import os
import sys
import json
import pyspark
from pyspark.sql import SparkSession
from random import random
import pandas as pd
import re
import numpy as np
import scipy
import pyspark.sql.functions as f
import time 
import datetime 
from datetime import datetime
import logging
import traceback
import yaml
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import col,lit
from datetime import datetime
# Custom Library
sys.path.insert(0,'/dbfs/mnt/mountdatalake/AZ_IDFCodebase/conf/')
import utilsShared
# Logger function
logger = utilsShared.getFormattedLogger(__name__)
logger.info("Performing Init Activities")
sys.path.insert(0,'/dbfs/mnt/mountdatalake/AZ_IDFCodebase/process-files-datavault/')
import utilsIO

2021-12-28 07:27:49,264 - __main__ - INFO - Performing Init Activities
2021-12-28 07:27:49,264 - __main__ - INFO - Performing Init Activities
2021-12-28 07:27:49,264 - __main__ - INFO - Performing Init Activities

In [0]:

envConfig = utilsShared.getEnvConf(logger,dbutils)
try:
  if "sqlserver" in envConfig.keys():
    jdbcHostname = envConfig['sqlserver']['jdbcHostname']
    jdbcDatabase = envConfig['sqlserver']['jdbcDatabase']
    jdbcPort = envConfig['sqlserver']['jdbcPort']
    username=envConfig['sqlserver']['username']
    password=envConfig['sqlserver']['password']
    hostNameInCertificate = envConfig['sqlserver']['hostNameInCertificate']
    jdbcUrl = envConfig['sqlserver']['jdbc-connection-string']
    jdbcUrl=jdbcUrl.format(jdbcHostname, jdbcPort, jdbcDatabase, username, password, hostNameInCertificate)
    #jdbcUrl= "jdbc:sqlserver://idfrgdemo.database.windows.net:1433;database=idfrgconfigdb;user=AccentureAdminLogin;password=Accenture2020!;encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;"
    connectionProperties = {"user" : "AccentureAdminLogin","password" : "Accenture2020!","driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
    print(jdbcUrl)
except:
  logger.info("Sql server properties are missing")
  raise ValueError("Sql server properties are missing "+ str(traceback.format_exc()))


2021-06-03 15:34:18,599 - __main__ - INFO - Reading Env Config from: /dbfs/mnt/mountdatalake/AZ_IDFCodebase/conf/envConfig.yaml
jdbc:sqlserver://idfrgdemo.database.windows.net:1433;database=idfrgconfigdb;user=AccentureAdminLogin;password=Accenture2020!;encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;

In [0]:
pushdown_query = "(select * from dbo.uijsoninput order by Modified_time desc OFFSET 0 ROWS) as t1"
df = spark.read.jdbc(url=jdbcUrl, table=pushdown_query, properties=connectionProperties)
df1 = df.withColumn("current_time",current_timestamp())
df2 = df1.withColumn("date_diff_min",(f.col("current_time").cast("long") - f.col("Modified_time").cast("long"))/60.).filter(col("date_diff_min") < 1)
null = None
isNull = True
targetPath = None
####### Modfied time dataobject##############################
for item in df2.collect():
  jsonObject = item.asDict()
  if 'ID' in jsonObject.keys():
    id= jsonObject['ID']
    #print(id)
  else:
    id="batch_id"
  if 'Modified_time' in jsonObject.keys():
    modified_time= jsonObject['Modified_time']
    modified_time1=datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
    #print(modified_time1)
  else:
    modified_time=datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
  batchtime=id+str(modified_time)
  baseconf={}
  commonconf={}
  commonconf['use-case']="DataIntegration"
  commonconf['prcs-name']="raw_to_curated"
  commonconf['logging-level']="DEBUG"
  baseconf['M0-common-conf']=commonconf
  for k,v in jsonObject.items():
    if k=="jsonValue":
      obj= json.loads(jsonObject['jsonValue'])
      for x in range(len(obj)):
        fobj=obj[x]
        #print(fobj)
        if fobj['type']=='Data Source Properties':
          logger.info("In Data Source Properties")
        elif fobj['type']=='Data Attribute Properties':
          logger.info("In Data Attribute Properties")
          var2='M3-ingestion-load'
          dataextract={}
          if 'sourceDataType' in fobj['fields'] and 'fileUpload' == fobj['fields']['sourceDataType']:
            if 'config'in fobj['fields']:
              if 'sourceList' in fobj['fields']['config']:
                for srclist in fobj['fields']['config']['sourceList']:
                  raw_path=fobj['fields']['config'][srclist]['path']
                  src_filename=fobj['fields']['config'][srclist]['file-name']
                  file_schema=[]
                  for datdef in fobj['fields']['config'][srclist]['cols']:
                    coldef={}
                    coldef['col_name']=datdef['columnName']
                    coldef['default_value']=null
                    coldef['isNullable']=isNull
                    if 'int64' == datdef['columnType']:
                      coldef['datatype']='int'
                    elif 'float64' == datdef['columnType']:
                      coldef['datatype']='float'
                    else:
                      coldef['datatype']=datdef['columnType']
                    file_schema.append(coldef)
              targetPath=fobj['fields']['attTargetPath']
              curatedPath=id + '/'
              file_extension=src_filename.split(".")[-1]
            #if file_extension == 'csv':
              #dataextract['type']='csv-batch-ingestion-process'
            if file_extension == 'json':
              dataextract['type']='json-batch-ingestion-process'
            elif file_extension == 'xml':
              dataextract['type']='xml-batch-ingestion-process'
            else:
              dataextract['type']='txt-batch-ingestion-process'
            dataextract['delimiter']=','
            dataextract['header']='true'
            dataextract['sampling-ratio-schema-validation']='0.05'
            dataextract['data-profile']='false'
            dataextract['raw-path']=raw_path[1:]
            dataextract['curated-path']=curatedPath
            dataextract['target-dir']=targetPath
            dataextract['target-format']='parquet'
            dataextract['comment']=''
            dataextract['skippedRows']='0'
            dataextract['src_filename']=src_filename
            dataextract['nullValue']='#MI'	
            dataextract['file_schema']=file_schema
          elif 'sourceDataType' in fobj['fields'] and 'tableUpload' == fobj['fields']['sourceDataType']:
            #print(fobj['fields'])
            targetPath=fobj['fields']['attTargetPath']
            curatedPath=fobj['fields']['attTargetZone']
            dataextract['databaseName']=fobj['fields']['databaseName']
            dataextract['tableName']=fobj['fields']['tableName']
            dataextract['target-format']='parquet'
            dataextract['curated-path']=curatedPath
            dataextract['target-dir']=targetPath
          baseconf[var2]=dataextract
        elif fobj['type']=='Data Curation Rules':
          logger.info("In Data Curation Rules")
          var3='M5-rules-engine-process'
          dataextract={}
          dataextract['type']='data-rules-engine-process'
          dataextract['multi-folder']='FALSE'
          dataextract['delimiter']='NA'
          dataextract['target-format']='parquet'
          data_rules=[]
          if 'dataset' in fobj['fields']:
            datasetType=fobj['fields']['dataset']
            #if datasetType == 'Remove Blank/NA':
                #blank = {}
                #blank['pattern'] = 'functional'
                #blank['with'] = 'dropna'
                #blank['col'] = ''
                #blank['newcol'] = ''
                #data_rules.append(blank)
          if 'curationRules' in fobj['fields']:
                x=json.loads(fobj['fields']['curationRules'])
                for rules in x:
                  if 'replace value' == rules["pattern"].lower():
                    y=json.loads(rules['rule'])
                    expression=y['value']
                    expressionName=y['name']
                    rep_val={}
                    rep_val['pattern'] ='replacevalue'
                    if 'Exact' in rules['columnType']:
                      rep_val['columnType'] = rules['columnType']
                      if rules['renameColumn'] == '':
                        rules['renameColumn'] = rules['columnName']
                      rep_val['newcol'] = rules['renameColumn']
                      rep_val['col'] = rules['columnName']
                    elif 'Alias' in rules['columnType']:
                      rep_val['columnType'] = rules['columnType']
                      rep_val['col'] = rules['aliasTypes']
                    else:
                      pass
                    rep_val['rule'] = expressionName
                    rep_val['with'] = rules['newValue']
                    rep_val['on'] = rules['existingValue'] 
                    rep_val['grp'] = rules['group']
                    data_rules.append(rep_val)
                  elif 'replace partial value' == rules["pattern"].lower():
                      y=json.loads(rules['rule'])
                      expression =y['value']
                      expressionName =y['name']
                      rep_par={}
                      rep_par['pattern'] = 'replacepartial'
                      if 'Exact' in rules['columnType']:
                        if rules['renameColumn'] == '':
                          rules['renameColumn'] = rules['columnName']
                        rep_par['columnType'] = rules['columnType']
                        rep_par['newcol'] = rules['renameColumn']
                        rep_par['col'] = rules['columnName']
                      elif 'Alias' in rules['columnType']:
                        rep_par['columnType'] = rules['columnType']
                        rep_par['col'] = rules['aliasTypes']
                      else:
                        pass
                      rep_par['rule'] = expressionName
                      rep_par['with'] = rules['newValue']
                      rep_par['on'] = rules['existingValue']
                      rep_par['grp'] = rules['group']
                      data_rules.append(rep_par)
                  elif 'compliance' == rules["pattern"].lower():
                    compl={}
                    compl['pattern'] = 'anonymization'
                    if 'Exact' in rules['columnType']:
                      compl['columnType'] = rules['columnType']
                      if rules['renameColumn'] == '':
                          rules['renameColumn'] = rules['columnName']
                      compl['newcol'] = rules['renameColumn']
                      compl['col'] = rules['columnName']
                    elif 'Alias' in rules['columnType']:
                      compl['columnType'] = rules['columnType']
                      compl['col'] = rules['aliasTypes']
                    else:
                      pass
                    y=json.loads(rules['rule'])
                    if y['name'] == 'Base 64':
                      y['name'] = 'base64'
                    compl['with'] = y['name']
                    data_rules.append(compl)
                  elif 'imputation' == rules["pattern"].lower():
                    #print(rules)
                    imput={}
                    imput['pattern'] = 'imputation'
                    if 'Exact' in rules['columnType']:
                      imput['columnType'] = rules['columnType']
                      if rules['renameColumn'] == '':
                          rules['renameColumn'] = rules['columnName']
                      imput['newcol'] = rules['renameColumn']
                      imput['col'] = rules['columnName']
                    elif 'Alias' in rules['columnType']:
                      imput['columnType'] = rules['columnType']
                      imput['col'] = rules['aliasTypes']
                    else:
                      pass
                    y=json.loads(rules['rule'])
                    imput['rule'] = y['name']
                    #imput['value'] =y['value']
                    data_rules.append(imput)
                  elif 'date standardization' == rules["pattern"].lower():
                    #print(rules)
                    dt_stand={}
                    dt_stand['pattern'] = 'datestandardization'
                    if 'Exact' in rules['columnType']:
                      dt_stand['columnType'] = rules['columnType']
                      if rules['renameColumn'] == '':
                          rules['renameColumn'] = rules['columnName']
                      dt_stand['newcol'] = rules['renameColumn']
                      dt_stand['col'] = rules['columnName']
                    elif 'Alias' in rules['columnType']:
                      dt_stand['columnType'] = rules['columnType']
                      dt_stand['col'] = rules['aliasTypes']
                    else:
                      pass
                    y=json.loads(rules['rule'])
                    dt_stand['rule'] = y['name']
                    data_rules.append(dt_stand)
                  elif 'functional' == rules["pattern"].lower():
                      fun = {}
                      y=json.loads(rules['rule'])
                      expression =y['value']
                      fun['pattern'] = 'functional'
                      if 'Exact' in rules['columnType']:
                        fun['columnType'] = rules['columnType']
                        if rules['renameColumn'] == '':
                          rules['renameColumn'] = rules['columnName']
                        fun['newcol'] = rules['renameColumn']
                        fun['col'] = rules['columnName'] 
                      elif 'Alias' in rules['columnType']:
                        fun['columnType'] = rules['columnType']
                        fun['col'] = rules['aliasTypes']
                      else:
                        pass
                      fun['rule']=y['name']
                      if 'lower case' in expression.lower():
                        fun['with'] ='lower'
                      elif 'upper case' in expression.lower():
                        fun['with'] = 'upper'
                      elif 'remove accent' in expression.lower():
                        fun['with'] = 'removeaccent'
                      elif 'accent remove' in expression.lower():
                        fun['with'] = 'accentremove'
                      elif 'prefix' in expression.lower():
                        fun['with'] = 'prefix'
                        fun['value'] =rules['functValue']
                      elif 'suffix' in expression.lower():
                        fun['with'] = 'suffix'
                        fun['value'] =rules['functValue']
                      elif 'concat' in expression.lower():
                        fun['with'] = 'concat'
                        fun['value'] =rules['functValue']
                      elif 'substring' in expression.lower():
                        fun['with'] = 'substring'
                        fun['start'] = rules['startPos']
                        fun['length'] = rules['length']
                      elif 'left trim' in expression.lower():
                        fun['with'] = 'ltrim'
                      elif 'right trim' in expression.lower():
                        fun['with'] = 'rtrim'
                      elif 'left padding' in expression.lower():
                        fun['with'] = 'lpad'
                        fun['value'] =rules['functValue']
                        fun['length'] =rules['length']
                      elif 'right padding' in expression.lower():
                        fun['with'] = 'rpad'
                        fun['value'] =rules['functValue']
                        fun['length'] =rules['length']
                      elif 'split' in expression.lower():
                        fun['with'] = 'split'
                        fun['value'] =rules['functValue']
                        fun['grp'] =rules['group']
                      elif 'trimdoublequotes' in expression.lower():
                        fun['with'] = 'trimdoublequotes'
                      elif 'camelcase' in expression.lower():
                        fun['with'] = 'camelcase'
                      elif 'base64' in expression.lower():
                        fun['with'] = 'base64'
                      elif 'dropduplicates' in expression.lower():
                        fun['with'] = 'dropduplicates'
                      elif 'dropna' in expression.lower():
                        fun['with'] = 'dropna'
                      elif 'rename' in expression.lower():
                        fun['with'] = 'rename'
                      elif 'drop' in expression.lower():
                        fun['with'] = 'drop'
                      data_rules.append(fun)
                  elif 'business rules' == rules["pattern"].lower():
                   #print(rules)
                    bus={}
                    bus['pattern'] = 'businessrules'
                    if 'Exact' in rules['columnType']:
                      bus['columnType'] = rules['columnType']
                      if rules['renameColumn'] == '':
                          rules['renameColumn'] = rules['columnName']
                      bus['newcol'] = rules['renameColumn']
                      bus['col'] = rules['columnName'] 
                    elif 'Alias' in rules['columnType']:
                      bus['columnType'] = rules['columnType']
                      bus['col'] = rules['aliasTypes']
                    else:
                      pass
                    y=json.loads(rules['rule'])
                    bus['rule']=y['name']
                    bus['value'] =rules['functValue']
                    data_rules.append(bus)
                  else:
                     pass
             
          dataextract['DATA-RULES']=data_rules
          baseconf[var3]=dataextract
         
#         elif fobj['type']=='Data Quality Checks':
#           logger.info("Data Quality Checks")
#           var4='M'+str(x)+'-data-quality-process'
#           dataextract={}
#           dataextract['type']='data-quality-batch-process'
#           dataextract['flag']=fobj['flag']
#           dataextract['tableName']=src_filename
#           baseconf[var4]=dataextract
        elif fobj['type']=='Deduplication':
          logger.info("Deduplication")
          #var5='M'+str(x)+'-data-deduplication-process'
          #dataextract={}
          #dataextract['type']=fobj['type']
          #dataextract['flag']=fobj['flag']
          #baseconf[var5]=dataextract
        
        else:
          logger.info("no Data Properties ")
          pass
      logger.info("end of range loop x")
    else:
      pass
  path='/dbfs/mnt/mountdatalake/UIconf/BatchProcessing/'
  if targetPath:
    file_name= id +"_"+targetPath+"_"+batchtime+".json"
    file=path+file_name
    with open(file, 'w') as fp:
      json.dump(baseconf, fp)
    logger.info("Archive File with ", file_name," is created")

    path='/dbfs/mnt/mountdatalake/AZ_IDFCodebase/data-conf-ingestion/'
    file_name= id +"_"+targetPath+"_rawtocurated.json"
    file=path+file_name
    with open(file, 'w') as fp:
      json.dump(baseconf, fp)
    logger.info("Config File with ", file_name," is created")
  
######## Modified time datobject#############################

In [0]:
pushdown_query = "(select TOP 1 * from dbo.uijsoninput order by Modified_time desc ) as t1"
df5 = spark.read.jdbc(url=jdbcUrl, table=pushdown_query, properties=connectionProperties)
df6 = df5.withColumn("current_time",current_timestamp())
null = None
isNull = True
####### Modfied time dataobject##############################
for item in df6.collect():
  jsonObject = item.asDict()
  if 'ID' in jsonObject.keys():
    id= jsonObject['ID']
  else:
    id="batch_id"
  if 'Modified_time' in jsonObject.keys():
    modified_time= jsonObject['Modified_time']
    modified_time1=datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
    #print(modified_time1)
  else:
    modified_time=datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
  batchtime=id+str(modified_time)
  baseconf={}
  commonconf={}
  commonconf['use-case']="DataIntegration"
  commonconf['prcs-name']="raw_to_curated"
  commonconf['logging-level']="DEBUG"
  baseconf['M0-common-conf']=commonconf
  for k,v in jsonObject.items():
    if k=="jsonValue":
      obj= json.loads(jsonObject['jsonValue'])
      for x in range(len(obj)):
          fobj=obj[x]
          if fobj['type']=='Data Quality Checks':
            #logger.info("In Data Quality Properties")
            #dataextract = {}
            if 'fromDQDashboard' in fobj['fields'] and fobj['fields']['fromDQDashboard'] == True:
              if 'dataset'in fobj['fields']:
                tmp_list1 = []
                for z in fobj['fields']['dataset']:
                  tmp_list1.append(z)

              if 'rules' in fobj['fields']:
                tmp_list2 = []
                for z in fobj['fields']['rules']:
                  tmp_list2.append(z)

              x = []
              b = []
              a = 0
              for i in tmp_list2:
                a = i['datasetId']
                x = i['dataQualityAssessment'] 
                for y in x:
                  z = {}
                  z = y
                  z['dataID'] = a
                  #z['rule'] = y['rule']
                  #z['']
                  b.append(z)

              c = {}
              obj_list = []
              for j in tmp_list1:
                for i in b:
                  if i['dataID'] == j['id']:
                    c = {**i,**j}
                    obj_list.append(c)
      #Creating dqmaster dataframe               
      pushdown_query = "(select * from dbo.dqMaster ) as t1"
      df_master = spark.read.jdbc(url=jdbcUrl, table=pushdown_query, properties=connectionProperties)
      df_master = df_master.withColumn("IsActive",lit(0)).cache()
      #df_master_bkp = df_master
      columns=df_master.columns
      #df_master.display()
      row_no = df_master.agg({"ROW_NO": "max"}).collect()[0][0]    
      row_no = int(row_no)+1
      empty_rdd = spark.sparkContext.emptyRDD()
      #dq_schema = StructType([StructField("RULE_ID", StringType()), StructField("STATS_DESC", StringType())])
      DF_JSON = spark.createDataFrame(empty_rdd, df_master.schema)     
      for i in obj_list:
        #Checking if key exists 
          if i["email_alert"] is True:
            i["email_alert"]=1
          else:
            i["email_alert"]=0
          if i["is_active"] is True:
            i["is_active"]=1
          else:
            i["is_active"]=0
          if 'remediate' not in i.keys() or i['remediate']['remediate'] is False:
              i['REMEDIATE_FLG'] = 0
              i['REM_RULE_TYPE'] = None
              i['REM_DEFAULT_VAL'] = 'NULL'
              i['REM_RULE_VAL'] = 'NULL'
              i['REM_METHOD'] = 'NULL'
          if 'UPLOAD_PATH' in i.keys():
               i['data_path'] = '/mnt/mountdatalake/' + i['UPLOAD_PATH']['data_path']
          if 'REF_PATH' in i.keys():
              i['ref_data_path'] = '/mnt/mountdatalake/' + i['REF_PATH']['ref_data_path']
          if 'REF_PATH' not in i.keys():
              i['ref_data_path'] = 'NULL'
          if 'column' not in i.keys() or len(i['column'])==0:
              i['column'] = 'NULL'
          elif type(i['column']) is str:
              i['column'] = i['column']
          elif type(i['column']) is list:
              i['column'] = ','.join(i['column'])
          if 'rule_description' not in i.keys():
            i['rule_description'] = 'NULL'
          if 'threshold_score' not in i.keys():
            i['threshold_score'] = 'NULL'
          if 'rule_val' not in i.keys():
            i['rule_val'] = 'NULL'
          if 'delimiter' not in i.keys():
            i['delimiter'] = ','
          if 'skippedRows' not in i.keys():
            i['skippedRows'] = '0'
          if 'nullValue' not in i.keys():
            i['nullValue'] = 'NULL'
          if 'dimension' not in i.keys():
            i['dimension'] = 'NULL'
          if 'email_address' not in i.keys():
            i['email_address'] = 'NULL'
            
          #Remediation
          if i['rule'] == 'isnull' and i['remediate']['remediate'] is True: #false
            remediation = i['remediate']['remediation']
            if remediation == "interpolate":
              i['REMEDIATE_FLG'] = 1
              i['REM_RULE_TYPE'] = remediation
              i['REM_DEFAULT_VAL'] = 'NULL'
              i['REM_RULE_VAL'] = 'NULL'
              i['REM_METHOD'] = i['remediate']["remidiation_rule"]["interpolation_method"]
            if remediation == "defaultValue":
              i['REMEDIATE_FLG'] = 1
              i['REM_RULE_TYPE'] = "default-value"
              i['REM_DEFAULT_VAL'] = i["remediate"]["remidiation_rule"]["default_value"]
              i['REM_RULE_VAL'] = 'NULL'
              i['REM_METHOD'] = 'NULL'
            if remediation == "mean":
              i['REMEDIATE_FLG'] = 1
              i['REM_RULE_TYPE'] = remediation
              i['REM_DEFAULT_VAL'] = 'NULL'
              i['REM_RULE_VAL'] = 'NULL'
              i['REM_METHOD'] = 'NULL'
            if remediation == "mostRecurringValue":
              i['REMEDIATE_FLG'] = 1
              i['REM_RULE_TYPE'] = remediation
              i['REM_DEFAULT_VAL'] = 'NULL'
              i['REM_RULE_VAL'] = 'NULL'
              i['REM_METHOD'] = 'NULL'
            if remediation == "median":
              i['REMEDIATE_FLG'] = 1
              i['REM_RULE_TYPE'] = remediation
              i['REM_DEFAULT_VAL'] = 'NULL'
              i['REM_RULE_VAL'] = 'NULL'
              i['REM_METHOD'] = 'NULL'
            if remediation == "trimmed_mean":
              i['REMEDIATE_FLG'] = 1
              i['REM_RULE_TYPE'] = remediation
              i['REM_DEFAULT_VAL'] = 'NULL'
              i['REM_RULE_VAL'] = i['remediate']['REM_RULE_VAL']
              i['REM_METHOD'] = 'NULL'
            if remediation == "regression":
              i['REMEDIATE_FLG'] = 1
              i['REM_RULE_TYPE'] = remediation
              i['REM_DEFAULT_VAL'] = 'NULL'
              i['REM_RULE_VAL'] = 'NULL'
              i['REM_METHOD'] = 'NULL'
            if remediation == "lookup":########needs clarity#######
              rv_f={}
              rv={}
              rv_f[i['remediate']['remidiation_rule']['lookup_join_column']] = i['remediate']['remidiation_rule']['ref_join_column']
              rv['REF_TABLE_NAME']=i['remediate']['remidiation_rule']['reference_table_name']
              rv['RULE_COL_TO_MATCH']=rv_f
              rv['REF_DEPENDENT_COL']=i['remediate']['remidiation_rule']['reference_column_name']
              i['REM_RULE_VAL']=rv
              i['REM_RULE_VAL']=str(i['REM_RULE_VAL']).replace('\'', '"')
              i['REMEDIATE_FLG'] = 1
              i['REM_RULE_TYPE'] = remediation
              i['REM_DEFAULT_VAL'] = 'NULL'
              i['REM_METHOD'] = 'NULL'
              i['ref_data_path'] = '/mnt/mountdatalake/' + i['remediate']['LOOKUP_PATH']['lookup_data_path']
                 
    #Custom expression
          if i['rule'] == 'value_pairs_to_be_equal':
            if i['rule_val']['data_type_enforcement_flag']==True:
              i['rule_val']['data_type_enforcement_flag']='true'
            else:
              i['rule_val']['data_type_enforcement_flag']='false'
              
          if i['rule'] == 'pair_values_a_to_be_greater_than_b':
            if i['rule_val']['data_type_enforcement_flag']==True:
              i['rule_val']['data_type_enforcement_flag']='true'
            else:
              i['rule_val']['data_type_enforcement_flag']='false'
            if i['rule_val']['date_type']==True:
              i['rule_val']['date_type']='true'
            else:
              i['rule_val']['date_type']='false'
              
          if i['rule'] == 'anomaly':
            operator = i['rule_val']['operator']
            mean = 'None'
            variance = 'None'
            threshold = 'None'
            if operator == 'IF':
              i['rule_val'] = operator
            if operator == 'KS':
              if 'mean' in i['rule_val'].keys():
                mean = i['rule_val']['mean']
              if 'variance' in i['rule_val'].keys():
                variance = i['rule_val']['variance']
              if 'threshold' in i['rule_val'].keys():
                threshold = i['rule_val']['threshold']
              i['rule_val'] = str(operator)+','+str(threshold)+','+str(mean)+','+str(variance)
          if i['rule'] == 'foreignkey':
            rv_f={}
            rv_f['column'] = i['column']
            rv_f['threshold_score'] = i['threshold_score']
            i['rule_val']=rv_f
          if i['rule'] == 'ksdistribution':
            rv = {}
            rv['p_value'] = i['rule_val']['p_val']
            rv['dist'] = i['rule_val']['dist']
            i['rule_val'] = rv
          if i['rule'] == 'latency-check':
            source_column_name = i['source_column']
            target_column_name = i['target_column']
            i['column'] = source_column_name+","+target_column_name
            source_date_format = 'DFLT'
            target_date_format = 'DFLT'
            sla = i['SLA']
            if 'source_datetime_format' in i.keys():   
              source_date_format = i['source_datetime_format']
            if 'target_datetime_format' in i.keys():   
              target_date_format = i['target_datetime_format']
            i['rule_val'] = str(sla)+"$"+str(source_date_format)+"$"+str(target_date_format)
          if i['rule'] == 'straight-look-up':
            rv_f={}
            rv={}
            rv_f[i['column']] = i['rule_val']['REF_COL_TO_MATCH']
            rv['REF_TABLE_NAME']=i['rule_val']['REF_TABLE_NAME']
            rv['RULE_COL_TO_MATCH']=rv_f
            rv['REF_DEPENDENT_COL']=i['rule_val']['REF_DEPENDENT_COL']
            i['rule_val']=rv  
          if i['rule'] == 'custom-expr':
            operator = "custom"
            if 'rule_val' in i.keys() and type(i['rule_val']) is dict:
              operator = i['rule_val']['operator']
            if operator == "BETWEEN":
              minsize = i['rule_val']['min_val']
              maxsize = i['rule_val']['max_val']
              col_name = i['column']
              i['rule_val'] = col_name + " " + operator + " " + str(minsize) + " AND " + str(maxsize)
            if operator == "IN":
              value = i['rule_val']['values']
              col_name = i['column']
              i['rule_val'] = col_name + " " + operator + " (" + value + ")"
            if operator == "NOT IN":
              value = i['rule_val']['values']
              col_name = i['column']
              i['rule_val'] = col_name + " " + operator + " (" + value + ")"
            if operator == "rlike":
              value = i['rule_val']['pattern']
              col_name = i['column']
              i['rule_val'] = col_name + " " + operator + " " + value
            if operator == "NOT rlike":
              value = i['rule_val']['pattern']
              col_name = i['column']
              i['rule_val'] = "~" + col_name + " " + operator + " " + value
            if operator == "like":
              value = i['rule_val']['pattern']
              col_name = i['column']
              i['rule_val'] = col_name + " " + operator + " " + value
            if operator == "NOT like":
              value = i['rule_val']['pattern']
              col_name = i['column']
              i['rule_val'] = "~" + col_name + " " + operator + " " + value
            if operator == ">":
              value = i['rule_val']['value']
              col_name = i['column']
              i['rule_val'] = col_name + " " + operator + " " + value
            if operator == "<":
              value = i['rule_val']['value']
              col_name = i['column']
              i['rule_val'] = col_name + " " + operator + " " + value
            if operator == ">=":
              value = i['rule_val']['value']
              col_name = i['column']
              i['rule_val'] = col_name + " " + operator + " " + value
            if operator == "<=":
              value = i['rule_val']['value']
              col_name = i['column']
              i['rule_val'] = col_name + " " + operator + " " + value
            if operator == "=":
              value = i['rule_val']['values']
              col_name = i['column']
              i['rule_val'] = col_name + " " + operator + " " + value
            
        #creating dataframe
          if 'rule_val' in i.keys():
            i['rule_val']=str(i['rule_val']).replace('\'', '"')
          dq_id = "DQ_0" + str(row_no)
          newRow = spark.createDataFrame([(str(row_no),str(i['rule_description']),str(i['rule']),str(i['dimension']),str(i["rule_val"]),str(i['data_domain']),str(i['column']),str(i['name']),str(i['data_path']),str(i['ref_data_path']),str(i['file_format']),bool(i['REMEDIATE_FLG']),str(i['REM_RULE_TYPE']),str(i['REM_DEFAULT_VAL']),str(i['REM_RULE_VAL']),str(i['REM_METHOD']),str(i["is_active"]),datetime.now(),str(i['threshold_score']),str(dq_id),str(id),bool(i['email_alert']),str(i['email_address']))], columns)
          #newRow.display()
          row_no=int(row_no)+1
          DF_JSON=DF_JSON.union(newRow)
      #df_master=df_master.filter(df_master.RULE_DIMENSION!=lit('Integrity'))
      df_master2=df_master.filter(df_master.BATCH_ID!=id).cache()
      #df_master_final = df_master2.union(DF_JSON)
      utilsIO.write_data_sql(DF_JSON,envConfig,'overwrite','dbo.dqMaster')
      utilsIO.write_data_sql(df_master2,envConfig,'append','dbo.dqMaster')
      
      #creating and writing DQ json
      dataquality={}
      dataquality['type']= "data-quality-batch-process"
      dataquality['dataset_name']= i["name"]
      dataquality['batch_id']=id
      dataquality['delimiter']=i['delimiter']
      dataquality['header']=i['header']
      dataquality['skippedRows']=i['skippedRows']
      dataquality['nullValue']=i['nullValue']
      baseconf['M6-data-quality-process']=dataquality
      path='/dbfs/mnt/mountdatalake/AZ_IDFCodebase/data-conf-ingestion/'
      file_name= id +"_"+i['name']+"_batchDataQuality.json"
      file=path+file_name
      with open(file, 'w') as fp:
        json.dump(baseconf, fp)
      logger.info("Config File with ", file_name," is created")

In [0]:
#### data mapping code ########
pushdown_query = "(select * from dbo.dataProvisioning order by Modified_time desc OFFSET 0 ROWS) as t1"
df = spark.read.jdbc(url=jdbcUrl, table=pushdown_query, properties=connectionProperties)
df3 = df.withColumn("current_time",current_timestamp())
df4 = df3.withColumn("date_diff_min",(f.col("current_time").cast("long") - f.col("Modified_time").cast("long"))/60.).filter(col("date_diff_min") < 1)
######## Modfied time dataobject##############################
for item in df4.collect():
  try:
    obj = item.asDict()
    m_count=0
    baseconf={}
    commonconf={}
    commonconf['use-case']="Data-Vault"
    commonconf['prcs-name']="clean_to_atomic"
    commonconf['logging-level']="DEBUG"
    baseconf['D'+str(m_count)+'-common-conf']= commonconf
    m_count=m_count+1
    if "jsonValue" in obj.keys():
      jobj= json.loads(obj['jsonValue'])
      if 'frequency' in jobj['mappingMetadata'].keys():
        if jobj['mappingMetadata']['frequency'] == 'Custom':
          frq='batch-date-single'
        else:
          frq=jobj['mappingMetadata']['frequency']
      else:
        frq='batch-date-single'
      ##### START source extract informations START ##################
      logger.info("start of source extract informations>>>"+frq)
      if 'sourceMapping' in jobj.keys():
        sobj=jobj['sourceMapping']
        for x in range(len(sobj)):
          vobj=sobj[x]
          if 'sourceName' in vobj.keys():
            src_name=vobj['sourceName']
            print(src_name)
          src_len=len(vobj)
          m_count=m_count+x
          var1='D'+str(m_count)+'-data-extract-process'
          dataextract={}
          sourceList=[]
          filters={}
          combines=[]
          uniones=[]
          dataextract['type']="data-extract-process"
          for k,v in vobj.items():
            if 'sourceFields' in k:
              logger.info("In sourceFields")
              #count=0
              count=1
              for z in v:
                var2='source'+str(count)
                sourceList.append(var2)
                dataextract['sourceList']=sourceList
                tmp={}
                if 'zone' in z.keys():
                  if z['zone'] == 'Raw':
                    tmp['type']='raw-zone'
                  elif z['zone'] == 'Curated':
                    tmp['type']='curated-zone'
                  elif z['zone'] == 'Enriched':
                    tmp['type']='enriched-zone'
                  else:
                    pass
                tmp['read-type']=frq
                if 'sourceType' in z.keys():
                  #tmp['source-type']=z['sourceType']
                  if z['sourceType'] == 'tableName':
                    tmp['table-name']=z['tableName']
                  elif z['sourceType'] == 'fileName':
                    if 'fileType' in z.keys():
                      if(len(z['fileType'])>0):
                        tmp['file-format']=z['fileType']
                    fileName = os.path.basename(z['fileName'])
                    path=os.path.dirname(z['fileName']) 
                    if(len(path)>0):
                      path=path+'/'
                      tmp['path']=path
                    if(len(fileName)>0):
                      tmp['fileName']=fileName
                  else:
                    pass
                if(len(z['sourceAttributes'])> 0):
                  tmp['cols']=list(z['sourceAttributes'].split(","))
                if(len(z['targetAttributes'])> 0):
                   tmp['alias']=list(z['targetAttributes'].split(","))
                else:
                  pass
                dataextract[var2]=tmp
                count=count+1
            if 'whereClauseFields' in k:
              logger.info("In whereClauseFields")
              #filters={}
              filters['filter-type']=str(v['operator']).lower()
              filters['tableName']=v['tableName']
              filters['value']=v['value']
              filters['colName']= list(v['attributes'].split(","))
              vcnt=0
              for q in v.get('whereClauseSubfields'):
                v1="key"+str(vcnt)
                tmp={}
                tmp['filter-type']=str(q['operator']).lower()
                tmp['condition']=q['condition']
                tmp['tableName']=q['tableName']
                if len(q['attributes']) > 0:
                  tmp['colName']=list(q['attributes'].split(","))
                tmp['value']=q['value']        
                filters[v1]=tmp
                vcnt=vcnt+1
            if 'configureFields' in k:
              logger.info("In configureFields")
              #combines=[]
              for z in v:
                tmp1={}
                tmp1['type']='join'
                tmp1['base-table-name']=z['tableName']
                if(len(z['attributes'])>0):
                  tmp1['base-table-key']=list(z['attributes'].split(","))
                else:
                  tmp1['base-table-key']=''
                tmp1['source-table-name']=z['conditionTableName']
                if(len(z['conditionAttributes'])>0):
                  tmp1['source-table-key']=list(z['conditionAttributes'].split(","))
                else:
                  tmp1['source-table-key']=''
                #tmp1['condition']=z['condition']
                if z['joinType'] == 'Left Outer Join':
                  tmp1['join-type']='left'
                elif z['joinType'] == 'Right Outer Join':
                  tmp1['join-type']='right'
                elif z['joinType'] == 'Self Join':
                  tmp1['join-type']='self'
                elif z['joinType'] == 'Inner Join':
                  tmp1['join-type']='inner'
                else:
                  pass
                combines.append(tmp1)
            if 'unionFields' in k:
              logger.info("In unionFields")
              for z in v:
                tmp1={}
                tmp1['type']='union'
                tmp1['source-table-name']=z['tableName']
                uniones.append(tmp1)         
          logger.info("end of k,v loop")  
          ### CHange in Filter ##############################################################
          if len(filters) > 0:
            filterBaseTable ={k:v for k,v in filters.items() if 'key' not in k}
    #             for item in dataextract["sourceList"][0:1]:
    #               base = item
    #             dataextract[base]["filter"] = filterBaseTable
            for item in dataextract["sourceList"]:
              base = item
              if 'table-name' in dataextract[base].keys():
                if dataextract[base]["table-name"] == filterBaseTable["tableName"]:
                    dataextract[base]["filter"] = filterBaseTable
              elif 'path' in dataextract[base].keys() and 'fileName' not in dataextract[base].keys():
                if dataextract[base]["path"] == filterBaseTable["tableName"]:
                  dataextract[base]["filter"] = filterBaseTable
              elif 'path' not in dataextract[base].keys() and 'fileName' in dataextract[base].keys():
                if dataextract[base]["fileName"] == filterBaseTable["tableName"]:
                  dataextract[base]["filter"] = filterBaseTable
              elif 'path' in dataextract[base].keys() and 'fileName' in dataextract[base].keys():
                if (dataextract[base]["path"]+dataextract[base]["fileName"]) == filterBaseTable["tableName"]:
                  dataextract[base]["filter"] = filterBaseTable
              else:
                pass

            # Add Filter in other tables
            filterOtherTable ={k:v for k,v in filters.items() if 'key' in k}

            for k,v in filterOtherTable.items():
              for item1 in dataextract["sourceList"]:
                #if dataextract[item1]['source-type'] == 'tableName':
                if 'table-name' in dataextract[item1].keys():
                  if dataextract[item1]["table-name"] == v["tableName"]:
                    dataextract[item1]["filter"] = {k:v}
                #elif dataextract[item1]['source-type'] == 'fileName':
                elif 'path' in dataextract[item1].keys() and 'fileName' not in dataextract[item1].keys():
                  if dataextract[item1]["path"] == v["tableName"]:
                    dataextract[item1]["filter"] = {k:v}
                elif 'path' not in dataextract[item1].keys() and 'fileName' in dataextract[item1].keys():
                  if dataextract[item1]["fileName"] == v["tableName"]:
                    dataextract[item1]["filter"] = {k:v}
                elif 'path' in dataextract[item1].keys() and 'fileName' in dataextract[item1].keys() :
                  if (dataextract[item1]["path"]+dataextract[item1]["fileName"]) == v["tableName"]:
                     dataextract[item1]["filter"] = {k:v}
                else:
                  pass
          else:
            pass
          #### filter adding in sourcess logic end ##########################################
          ### Change for Combine Opearation #################################################
          if len(combines)>0:
            for item in dataextract["sourceList"][-1:]:
                src = item
            #for item in dataextract[src]["combines"]:
            for item in combines:
              for item1 in dataextract["sourceList"]:
                if 'table-name' in dataextract[item1].keys():
                  if dataextract[item1]["table-name"] == item["source-table-name"]:
                    dataextract[item1]["combine"] = item
                elif 'path' in dataextract[item1].keys() and 'fileName' not in dataextract[item1].keys():
                  #print("inside only path")
                  if dataextract[item1]["path"] == item["source-table-name"]:
                    dataextract[item1]["combine"] = item
                elif 'path' not in dataextract[item1].keys() and 'fileName' in dataextract[item1].keys():
                  if dataextract[item1]["fileName"] == item["source-table-name"]:
                    dataextract[item1]["combine"] = item
                elif 'path' in dataextract[item1].keys() and 'fileName' in dataextract[item1].keys() :
                  if (dataextract[item1]["path"]+dataextract[item1]["fileName"]) == item["source-table-name"]:
                    dataextract[item1]["combine"] = item
                else:
                   pass
          else:
            pass
            #############################################################################
          ##
          ### Change for union Opearation #################################################
          if len(uniones)>0:
            for item in dataextract["sourceList"][-1:]:
                src = item
            #for item in dataextract[src]["combines"]:
            for item in uniones:
              for item1 in dataextract["sourceList"]:
                if 'table-name' in dataextract[item1].keys():
                  if dataextract[item1]["table-name"] == item["source-table-name"]:
                    if 'combine' in dataextract[item1].keys():
                      dataextract[item1]["combine1"] = item
                    else:
                      dataextract[item1]["combine"] = item
                elif 'path' in dataextract[item1].keys() and 'fileName' not in dataextract[item1].keys():
                  #print("inside only path")
                  if dataextract[item1]["path"] == item["source-table-name"]:
                    if 'combine' in dataextract[item1].keys():
                      dataextract[item1]["combine1"] = item
                    else:
                      dataextract[item1]["combine"] = item
                elif 'path' not in dataextract[item1].keys() and 'fileName' in dataextract[item1].keys():
                  if dataextract[item1]["fileName"] == item["source-table-name"]:
                    if 'combine' in dataextract[item1].keys():
                      dataextract[item1]["combine1"] = item
                    else:
                      dataextract[item1]["combine"] = item
                elif 'path' in dataextract[item1].keys() and 'fileName' in dataextract[item1].keys() :
                  if (dataextract[item1]["path"]+dataextract[item1]["fileName"]) == item["source-table-name"]:
                    if 'combine' in dataextract[item1].keys():
                      dataextract[item1]["combine1"] = item
                    else:
                      dataextract[item1]["combine"] = item
                else:
                   pass
          else:
            pass
            #############################################################################
          ##

          baseconf[var1]=dataextract
        logger.info("end of for x loop")
        logger.info("end of source extract informations>>>")
        #print(baseconf)
      else:
        pass
    ##### END source extract informations END ##################
    #print(baseconf)
    #######START Transformations START#########
      logger.info("start of transformations extract process>>>")
      #trnf_cnt=2
      m_count=m_count+1
      if 'transformations' in jobj.keys():
        tobj=jobj['transformations']
        varT1='D'+str(m_count)+'-data-transform-process'
        transformextract={}
        transformextract['type']='data-transform-process'
        transformList=[]
        tcount=1
        for p,q in tobj.items():
          transformextract['transform-list']=transformList
          if 'default' in p:
            var1="T"+str(tcount)
            transformList.append(var1)
            m={}
            m['type']='default-value'
            def1={}
            for v1 in range(len(q)):
              tmpobj=q[v1]
              tmpv={}
              tmpv['type']=tmpobj['columnType']
              tmpv['value']=tmpobj['value']
              tmpv['tableName']=tmpobj['tableName']
              def1[tmpobj['columnName']]=tmpv
              m['def']=def1
              transformextract[var1]=m
            tcount=tcount+1
          if 'columnAlias' in p:
            var1="T"+str(tcount)
            transformList.append(var1)
            x={}
            x['type']='col-alias'
            def1={}
            for v1 in range(len(q)):
              tmp={}
              #tmp['tableName']=q[v1]['tableName']
              tmp['newName']=q[v1]['newColumnName']
              tmp['newType']=q[v1]['newColumnType']
              def1[q[v1]['columnName']]=tmp
              x['def']=def1
              transformextract[var1]=x
            tcount=tcount+1
          if 'filterProcess' in p:
            var1="T"+str(tcount)
            transformList.append(var1)
            x={}
            x['type']='filter-process'
            def1={}
            for v1 in range(len(q)):
              h='key'+str(v1)
              def1[h]=q[v1]
              x['def']=def1
              transformextract[var1]=x
            tcount=tcount+1
          if 'derivedDataFrame' in p:
            var1="T"+str(tcount)
            transformList.append(var1)
            x={}
            x['type']='derived-dataframe-process'
            def1={}
            for v1 in range(len(q)):
              h='key'+str(v1)
              tmp={}
              #tmp['tableName']=q[v1]['tableName']
              tmp['type']='col-union'
              tmp['columnName']=q[v1]['columnName']
              tmp['sets-col']=list(q[v1]['columnSet'].split(","))
              def1[h]=tmp
              x['def']=def1
              transformextract[var1]=x
            tcount=tcount+1
          if 'renameColumns' in p:
            var1="T"+str(tcount)
            transformList.append(var1)
            x={}
            x['type']='rename-cols'
            def1={}
            for v1 in range(len(q)):
              #h='key'+str(v1)
              tmp={}
              tmp['table-name']=q[v1]['tableName']
              #def1[h]=tmp
              def1 = tmp
              x['def']=def1
              transformextract[var1]=x
            tcount=tcount+1
          if 'dropDuplicates' in p:
            var1="T"+str(tcount)
            transformList.append(var1)
            x={}
            x['type']='drop-duplicates'
            def1={}
            for v1 in range(len(q)):
              #h='key'+str(v1)
              tmp={}
              #tmp['tableName']=q[v1]['tableName']
              tmp['drop-duplicates-cols']=list(q[v1]['columnList'].split(","))   
              #def1[h]=tmp
              def1=tmp
              x['def']=def1
              transformextract[var1]=x
            tcount=tcount+1
          if 'hashKey' in p:
            var1="T"+str(tcount)
            transformList.append(var1)
            x={}
            x['type']='hash-key'
            def1={}
            for v1 in range(len(q)):
              #h='key'+str(v1)
              tmp={}
              #tmp['tableName']=q[v1]['pathName']
              tmp[q[v1]['columnName']]=q[v1]['keyName']
              #def1[h]=tmp
              def1=tmp
              x['def']=def1
            transformextract[var1]=x
            tcount=tcount+1
          if 'typeConversion' in p:
            var1="T"+str(tcount)
            transformList.append(var1)
            x={}
            x['type']='type-conversion'
            def1={}
            for v1 in range(len(q)):
              h='key'+str(v1)
              tmp={}
              #tmp['tableName']=q[v1]['tableName']
              if q[v1]['newColumnName'] == '':
                q[v1]['newColumnName'] = q[v1]['columnName']
              tmp['newColName']=q[v1]['newColumnName']
              tmp['dataType']=q[v1]['Datatype']
              tmp['colName']=q[v1]['columnName']
              #def1[q[v1]['columnName']]=tmp
              def1=tmp
              x['def']=def1
              transformextract[var1]=x
            tcount=tcount+1
          if 'formatDateToString' in p:
            var1="T"+str(tcount)
            transformList.append(var1)
            x={}
            x['type']='format-date-to-string'
            def1={}
            for v1 in range(len(q)):
              h='key'+str(v1)
              tmp={}
              #tmp['tableName']=q[v1]['tableName']
              if q[v1]['columnName'] == '':
                q[v1]['columnName'] = q[v1]['columnToFormat']
              tmp['colName']=q[v1]['columnName']
              tmp['colToFormat']=q[v1]['columnToFormat']
              tmp['replace-char']=q[v1]['replaceChar']
              def1[h]=tmp
              x['def']=def1
              transformextract[var1]=x
            tcount=tcount+1
          if 'aggregation' in p:
            var1="T"+str(tcount)
            transformList.append(var1)
            x={}
            x['type']='aggregation'
            def1={}
            for v1 in range(len(q)):
              h='key'+str(v1)
              tmpobj=q[v1]
              tmp={}
              if tmpobj['aggregationProcessType'] == 'groupBy':
                tmp['process']='group-by-w-agg'
                tmp['group_by_list']=list(tmpobj['groupByList'].split(","))
                tm={}
                if len(tmpobj['aggregationList'].split(","))>=1:
                  for z in tmpobj['aggregationList'].split(","):
                    tm[z]=tmpobj['aggregationType']
                tmp['agg_list']=tm
                tmp['agg_type']=tmpobj['aggregationType']
              elif tmpobj['aggregationProcessType'] == 'substract':
                tmp['process']='subtract'
                tmp['colName']=tmpobj['columnName']
                tmp['left_col_name']=tmpobj['leftColName'] 
                tmp['right_col_name']=tmpobj['rightColName']
              elif tmpobj['aggregationProcessType'] == 'product':
                tmp['process']='product'
                tmp['colName']=tmpobj['columnName']
                tmp['first_col']=tmpobj['firstColName']
                tmp['second_col']=tmpobj['secondColName']
              else :
                pass
              def1[h]=tmp
              x['def']=def1
              transformextract[var1]=x
            tcount=tcount+1
          if 'derivedKeyProcess' in p:
            var1="T"+str(tcount)
            transformList.append(var1)
            x={}
            x['type']='derived-key-process'
            def1={}
            for v1 in range(len(q)):
              tmpobj=q[v1]
              h='key'+str(v1)
              if tmpobj['processType']=="case_cond":
                #print("inside case_cond")
                tmpv={}
                tmpv['columnName']=tmpobj['newColName']
                tmpv['process']=tmpobj['processType']
                case_list={}
                cnt=0
                for clist in tmpobj.get('caseConditions'):
                  cvar='case'+str(cnt)
                  case_list[cvar]=clist
                  cnt=cnt+1
                tmpv['case_list']=case_list
                #x[h]=tmpv
                def1[h]=tmpv
                x['def']=def1
                transformextract[var1]=x
              if tmpobj['processType']=='concat_sep':
                tmpv={}
                tmpv['colName']=tmpobj['newColName']
                tmpv['process']='concat_sep'         ###tmpobj['processType']
                tmpv['cols']=list(tmpobj['columnList'].split(","))
                tmpv['seprator']=tmpobj['defaultValue']
                def1[h]=tmpv
                x['def']=def1
                transformextract[var1]=x  
              if tmpobj['processType']=='rename':
                tmpv={}
                tmpv['colTorename']=tmpobj['colToRename'] 
                tmpv['process']='rename'           ###tmpobj['processType']
                tmpv['colName']=''
                tmpv['new_name']=tmpobj['newColName']
                def1[h]=tmpv
                x['def']=def1
                transformextract[var1]=x
              if tmpobj['processType']=='new-copy':
                tmpv={}
                tmpv['copy-col']=tmpobj['ColToCopy']
                tmpv['colName']=tmpobj['newColName']
                tmpv['process']=tmpobj['processType']                                    
                def1[h]=tmpv
                x['def']=def1
                transformextract[var1]=x
              if tmpobj['processType']=='todate':
                tmpv={}
                if tmpobj['newColName'] == '':
                  tmpobj['newColName'] = tmpobj['colToFormat']
                tmpv['colName']=tmpobj['columnName'] 
                tmpv['process']='todate'                 ###tmpobj['processType']
                tmpv['colToFormat']=tmpobj['colToFormat']
                tmpv['dateformat']=tmpobj['dateFormat']
                def1[h]=tmpv
                x['def']=def1
                transformextract[var1]=x
              if tmpobj['processType']=='upper':
                tmpv={}
                if tmpobj['newColName'] == '':
                  tmpobj['newColName'] = tmpobj['colToUpper']
                tmpv['colName']=tmpobj['newColName'] 
                tmpv['process']='upper'   ###tmpobj['processType']
                tmpv['colToUpper']=tmpobj['colToUpper']
                def1[h]=tmpv
                x['def']=def1
                transformextract[var1]=x
              if tmpobj['processType']=='lower':
                tmpv={}
                if tmpobj['newColName'] == '':
                  tmpobj['newColName'] = tmpobj['colToLower']
                tmpv['colName']=tmpobj['newColName'] 
                tmpv['process']='lower'            ###tmpobj['processType']
                tmpv['colToLower']=tmpobj['colToLower']
                def1[h]=tmpv
                x['def']=def1
                transformextract[var1]=x
              if tmpobj['processType']=='Prefix':
                tmpv={}
                if tmpobj['newColName'] == '':
                  tmpobj['newColName'] = tmpobj['derivedName']
                tmpv['colName']=tmpobj['newColName'] 
                tmpv['process']=tmpobj['processType']
                tmpv['derivedName']=tmpobj['derivedName']
                tmpv['value']=tmpobj['value']
                def1[h]=tmpv
                x['def']=def1
                transformextract[var1]=x
              if tmpobj['processType']=='Suffix':
                tmpv={}
                if tmpobj['newColName'] == '':
                  tmpobj['newColName'] = tmpobj['derivedName']
                tmpv['colName']=tmpobj['newColName'] 
                tmpv['process']=tmpobj['processType']
                tmpv['derivedName']=tmpobj['derivedName']
                tmpv['suffix']=tmpobj['suffix']
                def1[h]=tmpv
                x['def']=def1
                transformextract[var1]=x
              if tmpobj['processType']=='Substring':
                tmpv={}
                if tmpobj['newColName'] == '':
                  tmpobj['newColName'] = tmpobj['derivedName']
                tmpv['substring_col']=tmpobj['derivedName']
                tmpv['colName']=tmpobj['newColName'] 
                tmpv['process']='substring'          #tmpobj['processType']
                tmpv['start_index']=tmpobj['startPosition']
                tmpv['length']=tmpobj['length']
                def1[h]=tmpv
                x['def']=def1
                transformextract[var1]=x
              if tmpobj['processType']=='Left Padding':
                tmpv={}
                if tmpobj['newColName'] == '':
                  tmpobj['newColName'] = tmpobj['derivedName']
                tmpv['colName']=tmpobj['newColName'] 
                #tmpv['process']=tmpobj['processType']
                tmpv['process']='lpad'
                tmpv['colToPad']=tmpobj['derivedName']
                tmpv['pad-char']=tmpobj['value']
                tmpv['length']=tmpobj['length']
                def1[h]=tmpv
                x['def']=def1
                transformextract[var1]=x
              if tmpobj['processType']=='Right Padding':
                tmpv={}
                if tmpobj['newColName'] == '':
                  tmpobj['newColName'] = tmpobj['derivedName']
                tmpv['colName']=tmpobj['newColName'] 
                tmpv['process']=tmpobj['processType']
                tmpv['colToPad']=tmpobj['derivedName']
                tmpv['pad-char']=tmpobj['value']
                tmpv['length']=tmpobj['length']
                def1[h]=tmpv
                x['def']=def1
                transformextract[var1]=x
              if tmpobj['processType']=='Split':
                tmpv={}
                if tmpobj['newColName'] == '':
                  tmpobj['newColName'] = tmpobj['derivedName']
                tmpv['colName']=tmpobj['newColName'] 
                tmpv['process']='split'        ###tmpobj['processType']
                tmpv['split-col-name']=tmpobj['derivedName']
                tmpv['delimiter']=tmpobj['value']
                #tmpv['delimiter']=tmpobj['group']
                def1[h]=tmpv
                x['def']=def1
                transformextract[var1]=x
              if tmpobj['processType']=='hash-key':
                tmpv={}
                if tmpobj['newColName'] == '':
                  tmpobj['newColName'] = tmpobj['derivedName']
                tmpv['colName']=tmpobj['newColName'] 
                tmpv['process']='hash-key'        ###tmpobj['processType']
                tmpv['hash-key']=list(tmpobj['columnList'].split(","))
                def1[h]=tmpv
                x['def']=def1
                transformextract[var1]=x
              if tmpobj['processType']=='not-null-list':            
                tmpv={}
                tmpv['colName']=tmpobj['newColName'] 
                tmpv['process']=tmpobj['processType']
                tmpv['cols']=list(tmpobj['columnList'].split(","))
                def1[h]=tmpv
                x['def']=def1
                transformextract[var1]=x
              if tmpobj['processType']=='new-default':
                tmpv={}
                tmpv['default-value']=tmpobj['defaultValue']
                tmpv['colName']=tmpobj['newColName']
                tmpv['process']=tmpobj['processType']             
                def1[h]=tmpv
                x['def']=def1
                transformextract[var1]=x
              if tmpobj['processType']=='default_value_to_key':
                tmpv={}
                tmpv['colName']=tmpobj['newColName']
                tmpv['col_list']=list(tmpobj['columnList'].split(","))
                tmpv['process']='default_value_to_key'          ###tmpobj['processType']             
                tmpv['default-value']=tmpobj['defaultValue']
                def1[h]=tmpv
                x['def']=def1
                transformextract[var1]=x
              ### added new transformation 
              if tmpobj['processType']=='LTrim':
                tmpv={}
                if tmpobj['newColName'] == '':
                  tmpobj['newColName'] = tmpobj['derivedName']
                tmpv['colName']=tmpobj['newColName'] 
                tmpv['process']=tmpobj['processType']
                tmpv['derivedColumn']=tmpobj['derivedColumn']
                def1[h]=tmpv
                x['def']=def1
                transformextract[var1]=x
              if tmpobj['processType']=='RTrim':
                tmpv={}
                if tmpobj['newColName'] == '':
                  tmpobj['newColName'] = tmpobj['derivedName']
                tmpv['colName']=tmpobj['newColName'] 
                tmpv['process']=tmpobj['processType']
                tmpv['derivedColumn']=tmpobj['derivedColumn']
                def1[h]=tmpv
                x['def']=def1
                transformextract[var1]=x
              if tmpobj['processType']=='Trim':
                tmpv={}
                if tmpobj['newColName'] == '':
                  tmpobj['newColName'] = tmpobj['derivedName']
                tmpv['colName']=tmpobj['newColName'] 
                tmpv['process']='trim-column'           ####tmpobj['processType']
                tmpv['prevColumn']=tmpobj['derivedColumn']
                def1[h]=tmpv
                x['def']=def1
                transformextract[var1]=x
              if tmpobj['processType']=='Cast':
                tmpv={}
                if tmpobj['newColName'] == '':
                  tmpobj['newColName'] = tmpobj['derivedName']
                tmpv['colName']=tmpobj['newColName'] 
                tmpv['process']=tmpobj['processType']
                tmpv['derivedColumn']=tmpobj['derivedColumn']
                tmpv['datatype']=tmpobj['dataType']
                def1[h]=tmpv
                x['def']=def1
                transformextract[var1]=x
              if tmpobj['processType']=='custom_expression':
                tmpv={}
                tmpv['colName']=tmpobj['newColName'] 
                tmpv['process']='replace-custom-string'
                tmpv['substring_val']=tmpobj['custom_expression']
                def1[h]=tmpv
                x['def']=def1
                transformextract[var1]=x
              else:
                pass
            tcount=tcount+1 
            ###add Transformations
        baseconf[varT1]=transformextract
        logger.info("end of p,q for loop")
      logger.info("end of transformations extract process>>>")
    else:
      pass
    ####### END Transformations END#########
    #print(baseconf)
    if 'targetMapping' in jobj.keys():
      tobjs=jobj['targetMapping']
      v1="D9-load-enrich-process"
      targetextract={}
      targetextract['type']='load-enrich-process'
      targetextract['mode-of-write']='append'
      targetextract['read-type']='full'
      targetextract['source-filter']=''
      targetextract['src-table']=''
      if 'tableName' in tobjs.keys():
        targetextract['target-table']=tobjs['tableName']
      if 'attributes' in tobjs.keys():
        #targetextract['target-col-list']=list(tobjs['attributes'].split(","))
        ##### new chnage #########
        f=tobjs['attributes'].split(",")
        targetschema={}
        targetcollist=[]
        for d in f:
          c={}
          v=d.replace(", ",",").split(" ")
          if len(v) > 1:
            c['colName']=v[0]
            c['type']=v[1]
            targetschema[v[0]]=c
            targetcollist.append(v[0])
          else:
            targetcollist.append(v[0])
        ######  new chnage for target schema ######
        targetextract['target-col-list']=targetcollist
        if not targetschema:
          targetextract['target-schema']=''
        else :
          targetextract['target-schema']=targetschema
      baseconf[v1]=targetextract
    else:
      pass
    #print(baseconf)
    batchid= obj['ID']
    current_datetime = datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
    batchid_time=batchid+str(current_datetime)
    logger.info("start of writing to json file in DataLakeprocess>>>"+batchid_time)
    ##### Write as Json in DataLake #######
    path='/dbfs/mnt/mountdatalake/UIconf/DataProvisioning/'
    file_name=batchid_time+".json"
    file=path+file_name
    with open(file, 'w') as fp:
       json.dump(baseconf, fp)
    logger.info("end of writing to json file in DataLakeprocess process>>>"+file)
    path='/dbfs/mnt/mountdatalake/AZ_IDFCodebase/data-conf-datavault/'
    file_name= src_name +"_"+str(targetextract['target-table']).replace(".", "_")+"_datavault.json"
    file=path+file_name
    with open(file, 'w') as fp:
      json.dump(baseconf, fp)
    logger.info("Config File with ", file_name," is created")
    
  except:
    logger.info("Json object detils are missing, please update ")
    raise ValueError("ERROR Json object deatils are missing "+ str(traceback.format_exc()))
  #### Write as Json end in DataLake #######

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1038708126544886> in <module> 
 1 #### data mapping code ######## 
 2 pushdown_query = "(select * from dbo.dataProvisioning order by Modified_time desc OFFSET 0 ROWS) as t1" 
 ----> 3 df = spark . read . jdbc ( url = jdbcUrl , table = pushdown_query , properties = connectionProperties ) 
 4 df3 = df . withColumn ( "current_time" , current_timestamp ( ) ) 
 5 df4 = df3 . withColumn ( "date_diff_min" , ( f . col ( "current_time" ) . cast ( "long" ) - f . col ( "Modified_time" ) . cast ( "long" ) ) / 60. ) . filter ( col ( "date_diff_min" ) < 1 ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in jdbc (self, url, table, column, lowerBound, upperBound, numPartitions, predicates, properties) 
 629 jpredicates = utils . toJArray ( gateway , gateway . jvm . java . lang . String , predicates ) 
 630 return self . _df ( self . _jreader . jdbc ( url , table , jpredicates , jprop ) ) 
 --> 631 return self . _df ( self . _jreader . jdbc ( url , table , jprop ) ) 
 632 
 633 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 125 def deco ( * a , ** kw ) : 
 126 try : 
 --> 127 return f ( * a , ** kw ) 
 128 except py4j . protocol . Py4JJavaError as e : 
 129 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o315.jdbc.
: com.microsoft.sqlserver.jdbc.SQLServerException: Parse error at line: 1, column: 16: The ORDER BY clause is not valid in views, inline functions, derived tables, sub-queries, and common table expressions, unless TOP or FOR XML is also specified.
	at com.microsoft.sqlserver.jdbc.SQLServerException.makeFromDatabaseError(SQLServerException.java:262)
	at com.microsoft.sqlserver.jdbc.SQLServerStatement.getNextResult(SQLServerStatement.java:1632)
	at com.microsoft.sqlserver.jdbc.SQLServerPreparedStatement.doExecutePreparedStatement(SQLServerPreparedStatement.java:600)
	at com.microsoft.sqlserver.jdbc.SQLServerPreparedStatement$PrepStmtExecCmd.doExecute(SQLServerPreparedStatement.java:522)
	at com.microsoft.sqlserver.jdbc.TDSCommand.execute(IOBuffer.java:7225)
	at com.microsoft.sqlserver.jdbc.SQLServerConnection.executeCommand(SQLServerConnection.java:3053)
	at com.microsoft.sqlserver.jdbc.SQLServerStatement.executeCommand(SQLServerStatement.java:247)
	at com.microsoft.sqlserver.jdbc.SQLServerStatement.executeStatement(SQLServerStatement.java:222)
	at com.microsoft.sqlserver.jdbc.SQLServerPreparedStatement.executeQuery(SQLServerPreparedStatement.java:444)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.resolveTable(JDBCRDD.scala:61)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRelation$.getSchema(JDBCRelation.scala:226)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:35)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:370)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:384)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$2(DataFrameReader.scala:373)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:373)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:258)
	at org.apache.spark.sql.